In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange,tqdm
import argparse
import xlrd
from datetime import datetime,timedelta
import dateparser
import re
import vcf
import scipy.stats as stats

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
def getloclist(wd,end,filter_out):
    loc_list = []
    EIRA_found = False
    for root, dirs, files in os.walk(wd):
        for file in files:
            if file.endswith(end) and not file.endswith(filter_out):
                loc = os.path.join(root, file)
                loc_list.append(loc) #get bim file location
#     if EIRA_found == True:""
#         print("EIRA detected!")
#     else:
#         print("Warning! No EIRA bim detected! Use first scanned cohort as ref!")
    return loc_list

In [72]:
def openfile(file,n=0,sep=','):
    if file.endswith((".csv",".CSV")):
        clinical = pd.read_csv(file,sep=sep,skiprows=n)
    elif file.endswith((".xlsx",".xls")):
        clinical = pd.read_excel(file,skiprows=n)
    elif file.endswith(".dta"):
        clinical = pd.read_stata(file)
    else:
#         raise ValueError("no clinical file detected!")
        clinical = pd.DataFrame()
    return clinical
def search(columns,patterns=["sjc","crp","sdai","cdai","mtx","Methotrexate",'ccp','acpa']):
    list_found = []
    for i in trange(len(columns)):
        for j in patterns:
            if j.lower() in columns[i].lower():
                print("Found pattern",j ,"in column num",i,columns[i])
                list_found.append(columns[i])
    return list_found
def search_folder(loc):
    list_found = []
    for root, dirs, files in os.walk(loc):
        for file in files:
            path = os.path.join(root, file)
            opened = openfile(path)
            print("openning file ",file)
            list_found.append(search(opened.columns))
    return list_found

In [6]:
def todatetime(a):
    b = dateparser.parse(a)
#     b = datetime.strptime(b,"%d-%b-%y")
    return b

In [7]:
def countRemission(fam):
    count_case = len(fam[fam[5]==2])
    count_control = len(fam[fam[5]==1])
    count_miss = len(fam[fam[5]==-9])+len(fam[fam[5]==0])
    plt.bar(["case",'control','missing'],[count_case,count_control,count_miss])
    return count_case,count_control,count_miss

In [44]:
loc_list = ['./Pipelinerun_withPC/EARTH_A/mergedChr/EA127genotypes_merged.fam', './Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged.fam', './Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.fam', './Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged.fam', './Pipelinerun_withPC/TwoCompDAS/RAMS_merged_2cDAS.fam', './Pipelinerun_withPC/Vienna/mergedChr/vienna_merged.fam', './Pipelinerun_withPC/SERA/mergedChr/sera_merged.fam']

### Earth_B

In [8]:
EarthB_fam = pd.read_csv('./Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged.fam',sep='\s',header=None)
EarthB_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,1019_EA3725,1019_EA3725,0,0,0,2
1,1043_EA3839,1043_EA3839,0,0,0,2
2,1066_EA4028,1066_EA4028,0,0,0,1
3,1090_EA3886,1090_EA3886,0,0,0,1
4,1117_EA4136,1117_EA4136,0,0,0,2
...,...,...,...,...,...,...
61,822_EA3956,822_EA3956,0,0,0,1
62,847_EA3718,847_EA3718,0,0,0,1
63,86_EA3946,86_EA3946,0,0,0,1
64,870_EA3829,870_EA3829,0,0,0,2


In [9]:
EarthB = openfile("/exports/reum/knevel_lab/EARTH/SetB/clinical/EA_Samples.xlsx",sep=';')
search(EarthB.columns,patterns=['ccp'])

100%|███████████████████████████████████████| 66/66 [00:00<00:00, 367140.67it/s]

Found pattern ccp in column num 46 CCP_Status


['CCP_Status']

In [10]:
EarthB

,EANo,Gender,DOB,PtAge,Visit_BL,Visit_MTX_Start,Rpt_Das_Date,DAS_Int,LatestMTXEnd,MTXEnded_InMTXPeriod,MTX_Duration,MTXEndReason,NonMTXDMARD_InMTXPeriod,NonMTXDMARDStart,HXQStart,HCQ,SLZStart,SLZ,LFNStart,LFN,NofStrdIMIA,DAS28ESR_BL,DAS28CRP_BL,GV_BL,Tender28_BL,Swollen28_BL,ESR_BL,CRP_BL,DAS28ESR_FU,DAS28CRP_FU,GlobalVAS,Tender28,Swollen28,ESR,CRP,dDASESR,dDASCRP,dDASESR_ratio,dDASCRP_ratio,EULAR,EULAR_CRP,Smoking,Alcohol,BMI,SBP,DBP,CCP_Status,RF_Status,EMS,ScanImpression,HAQ_BL,ESR_Remission,ESR_Low_Activity,CRP_Remission,CRP_Low_Activity,Boolean_Remission_ESR,Boolean_Remission_CRP,BL_2C_DAS,FU_2C_DAS,d2C_DAS,Symptom_Duration,HB,IgG,IgA,IgM,AST
0,EA3704,F,1949-02-28,68.666667,2017-10-31,2017-11-03,2018-03-22,4.733333,NaT,N,NaN,NaN,N,NaT,NaT,N,NaT,N,NaT,N,2.0,3.928928,3.219002,20,0,15,39,11,1.795435,1.265533,4.0,0,0,12.0,1.0,2.133494,1.953469,0.543022,0.606855,Good Response,Good Response,False,0.0,28.054110,168.0,111.0,Negative,Negative,60.0,NaN,1.125,Yes,Yes,Yes,Yes,No,Yes,5.363927,0.415888,4.948039,12.0,118.0,7.3,1.91,0.36,18
1,EA3714,F,1968-01-02,49.866667,2017-11-14,2017-11-21,2018-04-23,5.300000,NaT,N,NaN,NaN,Y,2017-11-21,2017-11-21,BL,NaT,N,NaT,N,NaN,3.531166,3.379957,80,0,1,21,16,1.224607,1.637398,7.0,0,0,5.0,4.0,2.306559,1.742559,0.653200,0.515557,Good Response,Good Response,False,5.0,34.090790,115.0,82.0,Positive,Positive,340.0,Definite inflammatory changes,0.750,Yes,Yes,Yes,Yes,Yes,Yes,2.699928,0.659167,2.040761,10.0,130.0,9.1,4.18,1.46,16
2,EA3718,M,1952-03-18,65.675000,2017-11-21,2017-11-28,2018-05-16,5.833333,NaT,N,NaN,NaN,Y,2017-11-28,2017-11-28,BL,NaT,N,NaT,N,1.0,6.228554,6.636743,40,24,22,10,18,3.368812,3.447998,62.0,1,0,16.0,18.0,2.859742,3.188745,0.459134,0.480468,Moderate Response,Moderate Response,False,0.0,42.716049,149.0,75.0,Positive,Negative,60.0,Definite inflammatory changes,2.125,No,No,No,No,No,No,6.457079,1.766663,4.690416,12.0,139.0,8.2,1.98,0.48,19
3,EA3719,F,1964-05-03,53.569444,2017-11-28,2017-12-05,2018-04-03,4.166667,NaT,N,NaN,NaN,N,2018-05-17,NaT,N,2018-05-17,N,NaT,N,1.0,2.666607,3.079398,50,1,1,5,4,1.310000,2.365398,19.0,1,0,2.0,4.0,1.356607,0.714000,0.508739,0.231864,Good Response,Moderate Response,False,14.0,24.287401,132.0,78.0,Positive,Positive,360.0,Definite inflammatory changes,0.750,Yes,Yes,Yes,Yes,No,No,1.000000,0.659167,0.340833,6.0,137.0,10.2,1.59,1.34,25
4,EA3720,M,1970-02-11,47.797222,2017-11-28,2017-11-28,2018-04-17,4.633333,NaT,N,NaN,NaN,Y,2017-11-28,2017-11-28,BL,NaT,N,NaT,N,1.0,6.071740,6.396487,80,15,6,20,57,2.215163,3.269357,27.0,2,4,2.0,4.0,3.856578,3.127130,0.635168,0.488882,Good Response,Moderate Response,True,0.0,23.114187,126.0,80.0,Positive,Positive,NaN,Definite inflammatory changes,2.000,Yes,Yes,No,No,No,No,4.885756,2.659167,2.226588,52.0,113.0,9.6,1.68,1.10,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,EA4201,F,1956-08-20,62.944444,2019-07-30,2019-08-06,2019-12-03,4.100000,NaT,N,NaN,NaN,N,NaT,NaT,N,NaT,N,NaT,N,NaN,5.400068,5.001422,70,10,2,25,11,3.358527,3.340528,20.0,4,1,11.0,6.0,2.041542,1.660894,0.378058,0.332084,Moderate Response,Moderate Response,False,0.0,32.769395,113.0,68.0,Negative,Negative,360.0,Definite inflammatory changes,2.300,No,No,No,No,No,No,2.905158,2.167546,0.737611,31.0,117.0,10.3,2.41,1.52,18
64,EA4208,F,1977-01-11,42.588889,2019-08-13,2019-08-20,2019-11-12,2.966667,NaT,N,NaN,NaN,N,NaT,NaT,N,NaT,N,NaT,N,1.0,3.646824,4.766654,80,7,4,2,5,2.386607,2.865033,50.0,1,0,5.0,5.0,1.260217,1.901621,0.345566,0.398942,Good Response,Good Response,True,2.0,34.367914,108.0,65.0,Negative,Negative,180.0,Definite inflammatory changes,NaN,Yes,Yes,No,Yes,No,No,3.075056,1.075056,2.000000,19.0,143.0,NaN,NaN,NaN,22
65,EA4209,F,1986-10-04,32.858333,2019-08-13,2019-10-07,2020-01-20,5.233333,2019-11-18,Y,1.366667,ADR,Y,2019-11-18,NaT,N,NaT,N,2019-

In [13]:
EarthB_fam['id']=EarthB_fam[0].str.split("_",expand=True)[1]
EarthB_fam

,0,1,2,3,4,5,id
0,1019_EA3725,1019_EA3725,0,0,0,2,EA3725
1,1043_EA3839,1043_EA3839,0,0,0,2,EA3839
2,1066_EA4028,1066_EA4028,0,0,0,1,EA4028
3,1090_EA3886,1090_EA3886,0,0,0,1,EA3886
4,1117_EA4136,1117_EA4136,0,0,0,2,EA4136
...,...,...,...,...,...,...,...
61,822_EA3956,822_EA3956,0,0,0,1,EA3956
62,847_EA3718,847_EA3718,0,0,0,1,EA3718
63,86_EA3946,86_EA3946,0,0,0,1,EA3946
64,870_EA3829,870_EA3829,0,0,0,2,EA3829


In [17]:
EarthB_ccp = EarthB.drop_duplicates('EANo',keep='last').loc[:,['EANo','CCP_Status']]
EarthB_ccp['CCP']=EarthB_ccp['CCP_Status'].map({'Negative':1,'Positive':2,'notAvailable':-9})
EarthB_ccp

,EANo,CCP_Status,CCP
0,EA3704,Negative,1
1,EA3714,Positive,2
2,EA3718,Positive,2
3,EA3719,Positive,2
4,EA3720,Positive,2
...,...,...,...
63,EA4201,Negative,1
64,EA4208,Negative,1
65,EA4209,Positive,2
66,EA4204,Positive,2


In [15]:
EarthB_ccp['CCP_Status'].unique()

array(['Negative', 'Positive'], dtype=object)

In [19]:
EarthB_merged = pd.merge(EarthB_fam,EarthB_ccp,how='inner',left_on='id',right_on='EANo')
EarthB_merged

,0,1,2,3,4,5,id,EANo,CCP_Status,CCP
0,1019_EA3725,1019_EA3725,0,0,0,2,EA3725,EA3725,Positive,2
1,1043_EA3839,1043_EA3839,0,0,0,2,EA3839,EA3839,Negative,1
2,1066_EA4028,1066_EA4028,0,0,0,1,EA4028,EA4028,Negative,1
3,1090_EA3886,1090_EA3886,0,0,0,1,EA3886,EA3886,Positive,2
4,1117_EA4136,1117_EA4136,0,0,0,2,EA4136,EA4136,Positive,2
...,...,...,...,...,...,...,...,...,...,...
61,822_EA3956,822_EA3956,0,0,0,1,EA3956,EA3956,Positive,2
62,847_EA3718,847_EA3718,0,0,0,1,EA3718,EA3718,Positive,2
63,86_EA3946,86_EA3946,0,0,0,1,EA3946,EA3946,Negative,1
64,870_EA3829,870_EA3829,0,0,0,2,EA3829,EA3829,Positive,2


In [21]:
EarthB_merged.loc[:,[0,1,2,3,4,'CCP']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/CCP_assoc/EARTH_setB_merged_ccp.fam",sep='\t',index=False,header=False)

### Madrid

In [22]:
Madrid = openfile("/exports/reum/knevel_lab/Madrid/clinical/Clinical_data_GWAS_patients_HUP.dta")
Madrid_additional = openfile("/exports/reum/knevel_lab/Scripts_Samantha/GWAS_runscripts/GWAS_scripts/input_files_used/madrid/Additional clincal data pacients GWAS HUP.xls")

In [23]:
search(Madrid.columns,['ccp'])
# ccp state negative?

100%|███████████████████████████████████████| 58/58 [00:00<00:00, 331430.02it/s]

Found pattern ccp in column num 54 ccpsn


['ccpsn']

In [25]:
search(Madrid_additional.columns,['ccp'])

100%|███████████████████████████████████████| 52/52 [00:00<00:00, 448773.27it/s]


[]

In [27]:
Madrid['ccpsn']

array([ 1.,  0., nan], dtype=float32)

In [28]:
Madrid_fam = pd.read_csv('./Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.fam',sep='\s',header=None)
Madrid_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,1018_ARC796,1018_ARC796,0,0,0,-9
1,1020_ARC552,1020_ARC552,0,0,0,1
2,1042_HCSC06,1042_HCSC06,0,0,0,-9
3,1044_ARC602,1044_ARC602,0,0,0,1
4,1068_ARC646,1068_ARC646,0,0,0,2
...,...,...,...,...,...,...
136,922_ARC560,922_ARC560,0,0,0,1
137,944_HCSC12,944_HCSC12,0,0,0,-9
138,946_ARC611,946_ARC611,0,0,0,1
139,970_ARC666,970_ARC666,0,0,0,2


In [122]:
Madrid_fam['id'] = Madrid_fam[0].str.replace('^.*_',"").str.replace("^.*C","").str.lstrip("0").astype(int)
Madrid_ccp = pd.merge(Madrid,Madrid_fam,how='right',left_on='codigo2',right_on='id')
Madrid_ccp

/tmp/ipykernel_1809188/306130966.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Madrid_fam['id'] = Madrid_fam[0].str.replace('^.*_',"").str.replace("^.*C","").str.lstrip("0").astype(int)


,codigo2,tabaco,fr,fechaini,sexo,cort_v1,dosis_mtx_v1,dosis_leflu_v1,dosis_antipal_v1,dosis_szp_v1,dosis_oro_v1,dosis_csa_v1,a_tnf_v1,aba_v1,toci_v1,ritu_v1,cort_v2,dosis_mtx_v2,dosis_leflu_v2,dosis_antipal_v2,dosis_szp_v2,dosis_oro_v2,dosis_csa_v2,a_tnf_v2,aba_v2,toci_v2,ritu_v2,vgepac_v3,vgemed_v3,nad_v3,nat_v3,vsg_v3,pcr_v3,cort_v3,dosis_mtx_v3,dosis_leflu_v3,dosis_antipal_v3,dosis_szp_v3,dosis_oro_v3,dosis_csa_v3,a_tnf_v3,aba_v3,toci_v3,ritu_v3,das28_v3,das28pcr_v3,das28cat_v3,das28pcrcat_v3,sdai_v3,cdai_v3,cort_v4,edadinienf,tevol_v1,tevol_v3,ccpsn,hupi_v3,hupicat_v3,cortic_1yr,0,1,2,3,4,5,id
0,796.0,0.0,0.0,2018-07-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,2.0,1.0,NaN,0.15,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.280000,NaN,Remision,7.000000,5.500000,0.0,78.717812,0.733333,NaN,NaN,2.0,Remision,2025.00000,1018_ARC796,1018_ARC796,0,0,0,-9,796
1,552.0,0.0,0.0,2012-02-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,25.0,1.0,1.0,37.0,0.30,1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.151642,3.083066,Act. moderada,Act. baja,10.400000,10.100000,0.0,77.271233,12.866667,24.900000,0.0,6.5,Act. moderada,493.09259,1020_ARC552,1020_ARC552,0,0,0,1,552
2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1042_HCSC06,1042_HCSC06,0,0,0,-9,6
3,602.0,2.0,1.0,2014-02-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,10.0,0.0,1.0,6.0,0.20,4.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.304232,2.405500,Remision,Act. baja,7.700000,7.500000,2.0,73.539726,2.866667,15.000000,0.0,4.0,Act. leve,1911.50000,1044_ARC602,1044_ARC602,0,0,0,1,602
4,646.0,2.0,1.0,2015-01-15,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.30,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.163730,1.459066,Remision,Remision,0.300000,0.000000,NaN,63.191780,6.433333,18.100000,1.0,1.0,Remision,0.00000,1068_ARC646,1068_ARC646,0,0,0,2,646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,560.0,2.0,1.0,2013-03-15,Hombre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,15.0,2.0,0.0,12.0,1.50,5.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.979394,3.198092,Act. baja,Act. baja,8.200000,6.700000,5.0,73.008217,4.066667,15.266666,0.0,5.5,Act. moderada,100.00000,922_ARC560,922_ARC560,0,0,0,1,560
137,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,944_HCSC12,944_HCSC12,0,0,0,-9,12
138,611.0,0.0,0.0,2014-07-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,3.0,0.0,0.0,4.0,0.10,4.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.320406,1.559533,Remision,Remision,2.900000,2.800000,0.0,80.093147,1.733333,14.566667,0.0,1.0,Remision,4078.75000,946_ARC611,946_ARC611,0,0,0,1,611
139,666.0,0.0,0.0,2015-08-15,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,15.0,0.0,1.0,33.0,0.14,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.147555,1.975169,Act. baja,Remision,5.640000,5.500000,0.0,67.884933,6.133333,18.266666,0.0,3.5,Act. leve,0.00000,970_ARC666,970_ARC666,0,0,0,2,666


In [123]:
Madrid_ccp['ccpsn'] = Madrid_ccp['ccpsn'].fillna(-9)
Madrid_ccp['CCP'] = Madrid_ccp['ccpsn'].map({1.:1,0.:2,-9:-9})

In [124]:
Madrid_ccp

,codigo2,tabaco,fr,fechaini,sexo,cort_v1,dosis_mtx_v1,dosis_leflu_v1,dosis_antipal_v1,dosis_szp_v1,dosis_oro_v1,dosis_csa_v1,a_tnf_v1,aba_v1,toci_v1,ritu_v1,cort_v2,dosis_mtx_v2,dosis_leflu_v2,dosis_antipal_v2,dosis_szp_v2,dosis_oro_v2,dosis_csa_v2,a_tnf_v2,aba_v2,toci_v2,ritu_v2,vgepac_v3,vgemed_v3,nad_v3,nat_v3,vsg_v3,pcr_v3,cort_v3,dosis_mtx_v3,dosis_leflu_v3,dosis_antipal_v3,dosis_szp_v3,dosis_oro_v3,dosis_csa_v3,a_tnf_v3,aba_v3,toci_v3,ritu_v3,das28_v3,das28pcr_v3,das28cat_v3,das28pcrcat_v3,sdai_v3,cdai_v3,cort_v4,edadinienf,tevol_v1,tevol_v3,ccpsn,hupi_v3,hupicat_v3,cortic_1yr,0,1,2,3,4,5,id,CCP
0,796.0,0.0,0.0,2018-07-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,2.0,1.0,NaN,0.15,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.280000,NaN,Remision,7.000000,5.500000,0.0,78.717812,0.733333,NaN,-9.0,2.0,Remision,2025.00000,1018_ARC796,1018_ARC796,0,0,0,-9,796,-9
1,552.0,0.0,0.0,2012-02-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,25.0,1.0,1.0,37.0,0.30,1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.151642,3.083066,Act. moderada,Act. baja,10.400000,10.100000,0.0,77.271233,12.866667,24.900000,0.0,6.5,Act. moderada,493.09259,1020_ARC552,1020_ARC552,0,0,0,1,552,2
2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,NaN,NaN,NaN,1042_HCSC06,1042_HCSC06,0,0,0,-9,6,-9
3,602.0,2.0,1.0,2014-02-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,10.0,0.0,1.0,6.0,0.20,4.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.304232,2.405500,Remision,Act. baja,7.700000,7.500000,2.0,73.539726,2.866667,15.000000,0.0,4.0,Act. leve,1911.50000,1044_ARC602,1044_ARC602,0,0,0,1,602,2
4,646.0,2.0,1.0,2015-01-15,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.30,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.163730,1.459066,Remision,Remision,0.300000,0.000000,NaN,63.191780,6.433333,18.100000,1.0,1.0,Remision,0.00000,1068_ARC646,1068_ARC646,0,0,0,2,646,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,560.0,2.0,1.0,2013-03-15,Hombre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,15.0,2.0,0.0,12.0,1.50,5.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.979394,3.198092,Act. baja,Act. baja,8.200000,6.700000,5.0,73.008217,4.066667,15.266666,0.0,5.5,Act. moderada,100.00000,922_ARC560,922_ARC560,0,0,0,1,560,2
137,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,NaN,NaN,NaN,944_HCSC12,944_HCSC12,0,0,0,-9,12,-9
138,611.0,0.0,0.0,2014-07-01,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,3.0,0.0,0.0,4.0,0.10,4.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.320406,1.559533,Remision,Remision,2.900000,2.800000,0.0,80.093147,1.733333,14.566667,0.0,1.0,Remision,4078.75000,946_ARC611,946_ARC611,0,0,0,1,611,2
139,666.0,0.0,0.0,2015-08-15,Mujer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,15.0,0.0,1.0,33.0,0.14,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.147555,1.975169,Act. baja,Remision,5.640000,5.500000,0.0,67.884933,6.133333,18.266666,0.0,3.5,Act. leve,0.00000,970_ARC666,970_ARC666,0,0,0,2,666,2


In [125]:
Madrid_ccp.loc[:,[0,1,2,3,4,'CCP']]

,0,1,2,3,4,CCP
0,1018_ARC796,1018_ARC796,0,0,0,-9
1,1020_ARC552,1020_ARC552,0,0,0,2
2,1042_HCSC06,1042_HCSC06,0,0,0,-9
3,1044_ARC602,1044_ARC602,0,0,0,2
4,1068_ARC646,1068_ARC646,0,0,0,1
...,...,...,...,...,...,...
136,922_ARC560,922_ARC560,0,0,0,2
137,944_HCSC12,944_HCSC12,0,0,0,-9
138,946_ARC611,946_ARC611,0,0,0,2
139,970_ARC666,970_ARC666,0,0,0,2


In [126]:
Madrid_ccp.loc[:,[0,1,2,3,4,'CCP']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/CCP_assoc/madrid_merged_ccp.fam",sep='\t',index=False,header=False)

### RAMS

In [45]:
RAMS_fam = pd.read_csv(loc_list[3],sep='\s',header=None)
RAMS_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,RAMS_1_RAMS_1,RAMS_1_RAMS_1,0,0,0,1
1,RAMS_10_RAMS_10,RAMS_10_RAMS_10,0,0,0,1
2,RAMS_100_RAMS_100,RAMS_100_RAMS_100,0,0,0,1
3,RAMS_102_RAMS_102,RAMS_102_RAMS_102,0,0,0,1
4,RAMS_103_RAMS_103,RAMS_103_RAMS_103,0,0,0,1
...,...,...,...,...,...,...
240,RAMS_89_RAMS_89,RAMS_89_RAMS_89,0,0,0,1
241,RAMS_91_RAMS_91,RAMS_91_RAMS_91,0,0,0,1
242,RAMS_93_RAMS_93,RAMS_93_RAMS_93,0,0,0,2
243,RAMS_95_RAMS_95,RAMS_95_RAMS_95,0,0,0,1


In [46]:
RAMS_pheno = openfile("/exports/reum/knevel_lab/RAMS/clinical/pheno.csv")
RAMS_pheno

,fid_anon,iid_anon,fupno,start_dose_mtx,steroids,vaspat,tjc28_value,sjc28_value,DAS28_CRP,sex,age_baseline,disease_duration_at_baseline_mths,logCRP,smoke_baseline
0,RAMS_1,RAMS_1,6 Mo,NaN,No,78.0,27.0,16.0,7.087547,NaN,NaN,NaN,2.793616,NaN
1,RAMS_1,RAMS_1,Baseline,15.0,No,77.0,24.0,11.0,6.549305,F,56.92845,17.311480,2.331172,current
2,RAMS_3,RAMS_3,Baseline,15.0,No,50.0,5.0,7.0,4.952939,M,43.01267,14.032790,3.610918,current
3,RAMS_3,RAMS_3,3 Mo,NaN,NaN,30.0,3.0,2.0,3.695989,NaN,NaN,NaN,2.639057,NaN
4,RAMS_3,RAMS_3,6 Mo,NaN,No,90.0,0.0,0.0,3.378795,NaN,NaN,NaN,3.218876,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,RAMS_526,RAMS_526,Baseline,7.5,Yes,26.0,2.0,0.0,3.047825,F,48.36700,4.524590,2.588516,never
913,RAMS_527,RAMS_527,3 Mo,NaN,NaN,24.0,5.0,0.0,3.193232,NaN,NaN,NaN,1.791759,NaN
914,RAMS_527,RAMS_527,Baseline,10.0,No,50.0,16.0,10.0,5.430471,M,77.39542,7.606557,1.791759,never
915,RAMS_527,RAMS_527,12 Mo,NaN,No,28.0,0.0,0.0,1.997033,NaN,NaN,NaN,1.791759,NaN


In [48]:
search(RAMS_pheno.columns,['ccp','acpa'])

100%|███████████████████████████████████████| 14/14 [00:00<00:00, 169711.72it/s]


[]

### Vienna

In [54]:
vienna_fam = pd.read_csv(loc_list[5],sep='\s',header=None)
vienna_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,10_1130820744,10_1130820744,0,0,0,2
1,1012_1041715812,1012_1041715812,0,0,0,2
2,1014_1131619222,1014_1131619222,0,0,0,2
3,1015_1150204122,1015_1150204122,0,0,0,1
4,1017_1041664226,1017_1041664226,0,0,0,1
...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9
402,988_1153942083,988_1153942083,0,0,0,1
403,989_1085427379,989_1085427379,0,0,0,2
404,990_1150146097,990_1150146097,0,0,0,2


In [59]:
Vienna = openfile("/exports/reum/knevel_lab/Vienna/clinical/Rachel_data_ready_to deliver.csv",sep=';')
Vienna = Vienna.drop_duplicates('SampleID_last',keep='last')
Vienna

,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge
3,both,2,12/6/2007,4,4/11/2007,F,"50,2",4/1/2007,",7",4/11/2007,",7",0,nein,False,1,ja,False,5,True,",42",22,31,3,33,",625",0,4,4,0,12,1,"3,71772971735082",MDA,"3,10751710521146","7,82",LDA,"7,4",LDA,0,0,8/2/2007,,,,,1,9,12/6/2007,9/1/2008,2,,7,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
7,both,4,3/10/2008,4,3/14/2007,W,"52,3",,,,,2,,False,1,ja,False,"6,25",True,",5",56,52,5,45,2,0,8,10,0,12,1,"5,12966537337247",HDA,"3,91695259877997","14,2",MDA,"13,7",MDA,0,0,3/14/2007,20,,,Mtx,4,"35,1",3/14/2007,1/29/2010,1,MTX,11,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,0,BOTH,1031467068,3
14,clin,19,4/12/2005,7,1/5/2004,F,"59,7",10/30/2003,"1,5",1/1/2004,"1,3",3,früher,False,1,ja,False,25,True,",45",16,30,27,35,1,2,7,9,2,0,,"4,23841263722848",MDA,"3,87130984486647","15,15",MDA,"14,7",MDA,0,0,2/18/2004,25,,,Mtx,6,"21,2",2/18/2004,11/16/2005,1,MTX,15,,W,GG,GG,AG,GG,GT,AT,TT,CC,DRB1*03:01:01:01,DRB1*07:01:01:01Â,0,3,0,BOTH,1125128271,3
20,clin,22,7/31/2003,6,4/5/2002,F,"52,6",11/1/1992,"10,8",12/1/1994,"8,7",2,,False,0,nein,False,,True,5,14,,,,"2,125",0,0,0,0,,,,,,,,,,0,,4/5/2002,"17,5",,,Mtx,5,"15,4",7/31/2002,11/6/2003,2,MTX,15,,,,,,,,,,,,,,1,0,BOTH,1015176427,3
26,clin,24,5/28/2014,6,3/19/2013,F,"40,9",2/1/2012,"2,3",4/17/2013,"1,1",2,,False,0,nein,False,,False,",06",11,7,0,0,0,0,0,0,0,255,,"1,77652669095886",REM,"1,22720130652846",",76",REM,",7",REM,1,1,,,,,Resochin,4,"21,4",5/22/2013,2/24/2015,1,csDMARD,14,,,,,,,,,,,,,,1,0,BOTH,1150211421,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,both,4221,7/27/2006,6,6/1/2005,F,"47,6",11/1/2004,"1,7",6/1/2005,"1,2",2,,False,1,ja,False,5,True,",14",19,4,0,2,",625",0,0,0,0,12,"51,3","2,11710728541651",REM,"1,3311687454474",",54",REM,",4",REM,1,1,6/2/2005,30,,,Mtx,6,"64,1",6/1/2005,9/6/2010,1,MTX,13,,W,GG,GG,,AG,GG,AT,TT,CC,DRB1*03:01:01:01,DRB1*07:01:01:01Â,0,3,0,BOTH,1085431327,3
2196,clin,4225,4/7/1998,2,1/13/1998,F,"58,7",1/1/1973,"25,3",1/1/1974,"24,3",2,,False,1,ja,False,5,True,",73",41,47,22,58,"2,125",2,2,2,2,12,,"4,44543983908642",MDA,"3,56779137772232","11,63",MDA,"10,9",MDA,0,0,,,,2000,Salazopyrin,1,"17,3",4/7/1998,9/9/1999,1,csDMARD,2,,W,GG,GG,AA,AG,GG,,TT,TT,DRB1*07:01:01:01,DRB1*16:01:01Â,0,3,0,BOTH,1071528505,3
2201,clin,4232,2/6/2003,5,11/14/2001,M,"59,1",4/1/2001,"1,9",11/14/2001,"1,2",2,,False,0,nein,False,,False,",76",15,20,18,14,",375",1,0,0,1,12,,"2,45563514077155",REM,"2,29463439317341","5,56",LDA,"4,8",LDA,1,1,8/1/2001,"22,5",,,Mtx,5,"110,1",11/14/2001,11/30/2010,1,MTX,14,,,,,,,,,,,,,,1,0,BOTH,1049757643,3
2206,clin,4236,4/20/2006,5,3/17/2005,F,"68,7",3/1/2003,"3,1",3/17/2005,"1,1",0,nein,True,1,ja,False,"2,5",True,"3,55",,0,0,0,2,0,0,0,0,,"2,3",,,"2,25503241381184","3,55",LDA,0,REM,0,0,3/17/2005,10,,,Mtx,3,"69,8",8/2/2005,4/26/2011,2,MTX,13,,W,GG,GG,,AA,GG,TT,TT,CT,DRB1*04:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1031466324,3


In [52]:
search(Vienna.columns,['ccp'])

100%|███████████████████████████████████████| 70/70 [00:00<00:00, 264505.66it/s]

Found pattern ccp in column num 30 CCP
Found pattern ccp in column num 39 ccp_pos


['CCP', 'ccp_pos']

In [55]:
vienna_fam['id'] = vienna_fam[0].str.split("_",expand=True)[1]
vienna_fam

,0,1,2,3,4,5,id
0,10_1130820744,10_1130820744,0,0,0,2,1130820744
1,1012_1041715812,1012_1041715812,0,0,0,2,1041715812
2,1014_1131619222,1014_1131619222,0,0,0,2,1131619222
3,1015_1150204122,1015_1150204122,0,0,0,1,1150204122
4,1017_1041664226,1017_1041664226,0,0,0,1,1041664226
...,...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9,1153937793
402,988_1153942083,988_1153942083,0,0,0,1,1153942083
403,989_1085427379,989_1085427379,0,0,0,2,1085427379
404,990_1150146097,990_1150146097,0,0,0,2,1150146097


In [135]:
vienna_merged=pd.merge(vienna_fam,Vienna,how='left',left_on='id',right_on='SampleID_last')
vienna_merged

,0,1,2,3,4,5,id,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge
0,10_1130820744,10_1130820744,0,0,0,2,1130820744,both,3147.0,1/15/2013,11.0,9/27/2011,W,"42,9",5/1/2011,"1,7",12/15/2011,"1,1",0.0,nein,False,0,nein,False,,False,",14",6,15,3,6,0,1,0,,,45,"123,3","1,74423162845964",REM,"1,7651687454474","2,94",REM,"2,8",LDA,1.0,1,12/15/2011,25,,,Mtx-Tocilizumab,6,"67,5",8/28/2012,3/14/2018,2,bDMARD in combination with csDMARDs,15.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,0.0,BOTH,1130820744,3.0
1,1012_1041715812,1012_1041715812,0,0,0,2,1041715812,clin,1614.0,10/7/2003,4.0,8/22/2002,F,"57,4",10/1/2001,2,6/1/2002,"1,4",1.0,ja,False,0,nein,False,,True,",5",2,8,4,2,0,1,0,0,1,216,,",877203026391962",REM,"1,9970334089221","2,7",REM,"2,2",REM,1.0,1,8/22/2002,,,3000,Salazopyrin,4,17,8/22/2002,1/13/2004,1,csDMARD,13.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1041715812,3.0
2,1014_1131619222,1014_1131619222,0,0,0,2,1131619222,both,326.0,12/17/2007,7.0,8/23/2006,W,"62,3",7/15/2006,"1,4",7/1/2006,"1,5",0.0,nein,False,1,ja,False,5,True,",12",8,26,6,12,",5",1,10,10,1,12,"1,5","3,87048456887018",MDA,"3,65872013942543","14,32",MDA,"14,2",MDA,0.0,0,8/23/2006,15,,,Mtx,6,34,10/19/2006,8/4/2009,1,MTX,15.0,1,W,GG,GG,AG,GG,GG,TT,CT,CT,DRB1*04:04:01,DRB1*13:01:01Â,1,3.0,0.0,BOTH,1131619222,3.0
3,1015_1150204122,1015_1150204122,0,0,0,1,1150204122,both,2018.0,5/12/2006,6.0,4/8/2005,M,"45,6",12/14/2004,"1,4",4/8/2005,"1,1",1.0,ja,False,1,ja,False,5,False,",9",25,4,6,10,",375",2,1,1,2,115,"7,4","3,26519287487221",MDA,"2,80091043094232","4,9",LDA,4,LDA,1.0,0,4/8/2005,20,,,Enbrel-Mtx,2,"51,8",2/3/2006,5/7/2010,2,TNFi in combination with csDMARDs,13.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1150204122,3.0
4,1017_1041664226,1017_1041664226,0,0,0,1,1041664226,clin,2829.0,8/4/2009,10.0,4/10/2008,W,"77,9",1/15/2007,"2,6",5/1/2007,"2,3",0.0,nein,False,1,ja,False,"6,25",True,,,55,43,57,",375",7,3,3,7,,,,,,,,"19,8",MDA,1.0,1,4/10/2008,,,,Adalimumab,1,"5,8",8/4/2009,1/25/2010,4,TNFi mono,15.0,5,W,GG,GG,AG,GG,GG,AT,CT,TT,DRB1*01:01:01,DRB1*04:01:01Â,2,3.0,0.0,BOTH,1041664226,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9,1153937793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,988_1153942083,988_1153942083,0,0,0,1,1153942083,both,2763.0,4/20/2015,5.0,1/8/2014,F,"51,4",6/1/2013,"1,9",1/8/2014,"1,3",0.0,nein,False,0,nein,False,,False,"1,43",37,20,2,12,",625",0,1,1,0,12,",4","3,36764253885096",MDA,"2,78202701822342","4,63",LDA,"3,2",LDA,0.0,0,6/6/2014,20,,,Mtx,4,"13,6",6/6/2014,7/20/2015,1,MTX,15.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1153942083,3.0
403,989_1085427379,989_1085427379,0,0,0,2,1085427379,clin,2894.0,11/25/2009,5.0,9/25/2008,W,"42,8",12/1/2007,2,9/25/2008,"1,2",0.0,nein,False,0,nein,False,,False,",11",16,3,0,1,0,0,2,2,0,259,75,"2,77477170049678",LDA,"2,06105703903151","2,41",REM,"2,3",REM,1.0,1,9/5/2008,30,,,Mtx,5,"125,7",9/25/2008,1/23/2019,1,MTX,14.0,2,W,GG,GT,AG,AA,GG,AA,CT,CC,DRB1*01:01:01,DRB1*1

In [136]:
vienna_merged['ccp'] = vienna_merged['ccp_pos'].map({'0':1,'1':2,'notAvailable':-9})
vienna_merged['ccp'] = vienna_merged['ccp'].fillna(-9)
vienna_merged['ccp'] = vienna_merged['ccp'].astype(int)
vienna_merged

,0,1,2,3,4,5,id,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge,ccp
0,10_1130820744,10_1130820744,0,0,0,2,1130820744,both,3147.0,1/15/2013,11.0,9/27/2011,W,"42,9",5/1/2011,"1,7",12/15/2011,"1,1",0.0,nein,False,0,nein,False,,False,",14",6,15,3,6,0,1,0,,,45,"123,3","1,74423162845964",REM,"1,7651687454474","2,94",REM,"2,8",LDA,1.0,1,12/15/2011,25,,,Mtx-Tocilizumab,6,"67,5",8/28/2012,3/14/2018,2,bDMARD in combination with csDMARDs,15.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,0.0,BOTH,1130820744,3.0,2
1,1012_1041715812,1012_1041715812,0,0,0,2,1041715812,clin,1614.0,10/7/2003,4.0,8/22/2002,F,"57,4",10/1/2001,2,6/1/2002,"1,4",1.0,ja,False,0,nein,False,,True,",5",2,8,4,2,0,1,0,0,1,216,,",877203026391962",REM,"1,9970334089221","2,7",REM,"2,2",REM,1.0,1,8/22/2002,,,3000,Salazopyrin,4,17,8/22/2002,1/13/2004,1,csDMARD,13.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1041715812,3.0,2
2,1014_1131619222,1014_1131619222,0,0,0,2,1131619222,both,326.0,12/17/2007,7.0,8/23/2006,W,"62,3",7/15/2006,"1,4",7/1/2006,"1,5",0.0,nein,False,1,ja,False,5,True,",12",8,26,6,12,",5",1,10,10,1,12,"1,5","3,87048456887018",MDA,"3,65872013942543","14,32",MDA,"14,2",MDA,0.0,0,8/23/2006,15,,,Mtx,6,34,10/19/2006,8/4/2009,1,MTX,15.0,1,W,GG,GG,AG,GG,GG,TT,CT,CT,DRB1*04:04:01,DRB1*13:01:01Â,1,3.0,0.0,BOTH,1131619222,3.0,1
3,1015_1150204122,1015_1150204122,0,0,0,1,1150204122,both,2018.0,5/12/2006,6.0,4/8/2005,M,"45,6",12/14/2004,"1,4",4/8/2005,"1,1",1.0,ja,False,1,ja,False,5,False,",9",25,4,6,10,",375",2,1,1,2,115,"7,4","3,26519287487221",MDA,"2,80091043094232","4,9",LDA,4,LDA,1.0,0,4/8/2005,20,,,Enbrel-Mtx,2,"51,8",2/3/2006,5/7/2010,2,TNFi in combination with csDMARDs,13.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1150204122,3.0,1
4,1017_1041664226,1017_1041664226,0,0,0,1,1041664226,clin,2829.0,8/4/2009,10.0,4/10/2008,W,"77,9",1/15/2007,"2,6",5/1/2007,"2,3",0.0,nein,False,1,ja,False,"6,25",True,,,55,43,57,",375",7,3,3,7,,,,,,,,"19,8",MDA,1.0,1,4/10/2008,,,,Adalimumab,1,"5,8",8/4/2009,1/25/2010,4,TNFi mono,15.0,5,W,GG,GG,AG,GG,GG,AT,CT,TT,DRB1*01:01:01,DRB1*04:01:01Â,2,3.0,0.0,BOTH,1041664226,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9,1153937793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9
402,988_1153942083,988_1153942083,0,0,0,1,1153942083,both,2763.0,4/20/2015,5.0,1/8/2014,F,"51,4",6/1/2013,"1,9",1/8/2014,"1,3",0.0,nein,False,0,nein,False,,False,"1,43",37,20,2,12,",625",0,1,1,0,12,",4","3,36764253885096",MDA,"2,78202701822342","4,63",LDA,"3,2",LDA,0.0,0,6/6/2014,20,,,Mtx,4,"13,6",6/6/2014,7/20/2015,1,MTX,15.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1153942083,3.0,1
403,989_1085427379,989_1085427379,0,0,0,2,1085427379,clin,2894.0,11/25/2009,5.0,9/25/2008,W,"42,8",12/1/2007,2,9/25/2008,"1,2",0.0,nein,False,0,nein,False,,False,",11",16,3,0,1,0,0,2,2,0,259,75,"2,77477170049678",LDA,"2,06105703903151","2,41",REM,"2,3",REM,1.0,1,9/5/2008,30,,,Mtx,5,"125,7",9/25/2008,1/23/2019,1,MTX,14.0,2,W,GG,GT,AG,AA,GG,AA,CT,

In [137]:
vienna_merged.loc[:,[0,1,2,3,4,'ccp']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/CCP_assoc/vienna_merged_ccp.fam",sep='\t',index=False,header=False)

### SERA

In [71]:
SERA_fam = pd.read_csv(loc_list[6],sep='\s',header=None)
SERA_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,1_215284670,1_215284670,0,0,0,-9
1,100_215285060,100_215285060,0,0,0,1
2,1005_215284880,1005_215284880,0,0,0,2
3,1006_215293700,1006_215293700,0,0,0,-9
4,1007_215287480,1007_215287480,0,0,0,-9
...,...,...,...,...,...,...
663,984_215294600,984_215294600,0,0,0,2
664,985_215299420,985_215299420,0,0,0,2
665,986_215294170,986_215294170,0,0,0,2
666,987_215292670,987_215292670,0,0,0,-9


In [73]:
search_folder("/exports/reum/knevel_lab/SERA/Data/")

openning file  webEmploy.CSV


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 60349.70it/s]


openning file  webSteroidInj.CSV


100%|█████████████████████████████████████████| 9/9 [00:00<00:00, 110376.42it/s]


openning file  .DS_Store


...

100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 82782.32it/s]


openning file  webDisease.CSV


100%|████████████████████████████████████████| 34/34 [00:00<00:00, 42429.73it/s]

Found pattern crp in column num 4 CRP
Found pattern ccp in column num 12 CCPMeasureType
Found pattern ccp in column num 13 CCPFigure
Found pattern ccp in column num 14 CCPTitrePosNeg
Found pattern ccp in column num 15 CCPTitreUpper
Found pattern ccp in column num 16 CCPTiteLower
Found pattern crp in column num 19 IsCRPDoneYN
Found pattern ccp in column num 21 IsCCPDoneYN
Found pattern ccp in column num 23 CCPFigureOperator
Found pattern crp in column num 24 CRPFigureOperator
Found pattern crp in column num 29 CRPNAYN
Found pattern ccp in column num 31 CCPNAYN


[[],
 [],
 [],
 [],
 [],
 ['sjc28'],
 [],
 ['mtxTreatment'],
 ['DAS28CRP'],
 [],
 [],
 [],
 [],
 [],
 ['CRP',
  'CCPMeasureType',
  'CCPFigure',
  'CCPTitrePosNeg',
  'CCPTitreUpper',
  'CCPTiteLower',
  'IsCRPDoneYN',
  'IsCCPDoneYN',
  'CCPFigureOperator',
  'CRPFigureOperator',
  'CRPNAYN',
  'CCPNAYN']]

In [76]:
webDisease = openfile("/exports/reum/knevel_lab/SERA/Data/webDisease.CSV")
webDisease = webDisease.drop_duplicates(['DiseaseSNo'],keep='last')
webDisease

,DiseaseSNo,DiseaseVisitID,ESR,DasVAS,CRP,GlobalVAS,PainVAS,RFMeasureType,RFFigure,RFTitrePosNeg,RFTitreUpper,RFTitreLower,CCPMeasureType,CCPFigure,CCPTitrePosNeg,CCPTitreUpper,CCPTiteLower,MrnStffnssID,IsESRDoneYN,IsCRPDoneYN,IsRFDoneYN,IsCCPDoneYN,RFFigureOperator,CCPFigureOperator,CRPFigureOperator,IsPainVASDoneYN,IsGlobalVASDoneYN,IsMrnStiffnessDoneYN,ESRNAYN,CRPNAYN,RFNAYN,CCPNAYN,PainVASNAYN,GlobalVASNAYN
2,100,Month 6,7.0,2,1.1,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
5,101,Month 6,12.0,1,2.6,10.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
7,102,Month 6,5.0,0,0.7,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
10,103,Month 6,100.0,47,97.0,70.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
13,104,Month 6,7.0,86,1.4,50.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,1315,Month 6,5.0,25,6.0,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
2694,1316,Month 6,NaN,20,0.7,10.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
2696,1317,Month 6,NaN,65,NaN,25.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,Yes,No,No,No,No
2698,1318,Month 6,NaN,5,4.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,<,Yes,Yes,Yes,No,No,No,No,No,No


In [81]:
webDisease['CCPNAYN'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [82]:
def processdcID(row):
    row = str(row).replace(" ","").lstrip("0")
    return row
def processFAM(row):
    row = str(row).split("_")[1]
    return row

### Tacera

In [85]:
Tacera = openfile("/exports/reum/CKe/Tacera/RA-MAP_data_clinical.csv")
keyfile = openfile("/exports/reum/CKe/Tacera/OpenPseudonymised_RA-MAP DNA_IDs deidentified to date 02Dec16_with_HLA.xlsx")
Tacera_fam = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL.fam",sep='\s',header=None)

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [87]:
search(Tacera.columns,['acpa'])

100%|█████████████████████████████████████| 300/300 [00:00<00:00, 325139.84it/s]

Found pattern acpa in column num 1 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Unit\
Found pattern acpa in column num 2 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Value\
Found pattern acpa in column num 3 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\
Found pattern acpa in column num 4 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Unit\
Found pattern acpa in column num 5 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Value\
Found pattern acpa in column num 6 \RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\
Found pattern acpa in column num 7 \RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Unit SCR\
Found pattern acpa in column num 8 \RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Value SCR\
Found pattern acpa in column num 9 \RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\
Found pattern acpa in column num 13 \RAMAP\RAMAP\Clinical Data\00 Bas

['\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\ACPA\\ACPA Unit\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\ACPA\\ACPA Value\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\ACPA\\ACPA positive\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\Rheumatoid Factor\\RF Unit\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\Rheumatoid Factor\\RF Value\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\Rheumatoid Factor\\Rheum Factor positive\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.Screening\\ACPA\\ACPA Unit SCR\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.Screening\\ACPA\\ACPA Value SCR\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\.Screening\\ACPA\\ACPA positive SCR\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\00 Baseline\\ACPA\\ACPA Unit 00BL\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\00 Baseline\\ACPA\\ACPA Value 00BL\\',
 '\\RAMAP\\RAMAP\\Clinical Data\\00 Baseline\\ACPA\\ACPA positive 00BL\\']

In [89]:
Tacera['\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\ACPA\\ACPA positive\\'].unique()

array(['Yes', 'No', nan], dtype=object)

In [90]:
Tacera = Tacera.merge(keyfile,how='left',left_on="Subject ID",right_on='Digest')

In [91]:
Tacera

,Subject ID,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Unit\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Value\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Unit\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Value\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Unit SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Value SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Unit SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Value SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Unit 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA positive 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Basophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Date of FBC 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Eosinophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\GL 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Gdl 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\HB 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Lymphocytes 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Monocytes 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Neutrophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\PLT 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\WBC 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\CRP Value (mg/L) 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\ESR Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Evaluator global 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Fatique 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Outside 2 week window 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Pain score 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Patient global 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total 66/68 Swollen Joints 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total 66/68 Tender Joints 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total Swollen 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total Tender 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Use of sign '<' 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS-28 CRP 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS28 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS28 relative 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\HAQ\Metrologist's score 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\RF Unit 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\RF Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\Rheum Factor positive 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\SDAI\SDAI <= 3.3 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\SDAI\SDAI score 00BL\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\CRP Value (mg/L) 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\ESR Value 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Evaluator global 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Fatique 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Outside 2 week window 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Pain score 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Patient global 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total 66/68 Swollen Joints 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total 66/68 Tender Joints 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total Swollen 03M\,

In [94]:
Tacera_fam

,0,1,2,3,4,5
0,D-12945-1_D-12945-1,D-12945-1_D-12945-1,0,0,0,-9
1,D-21657-1_D-21657-1,D-21657-1_D-21657-1,0,0,1,1
2,D-21658-1_D-21658-1,D-21658-1_D-21658-1,0,0,2,2
3,D-21659-1_D-21659-1,D-21659-1_D-21659-1,0,0,2,2
4,D-21660-1_D-21660-1,D-21660-1_D-21660-1,0,0,2,1
...,...,...,...,...,...,...
290,D-23461-1_D-23461-1,D-23461-1_D-23461-1,0,0,0,-9
291,D-33052-1_D-33052-1,D-33052-1_D-33052-1,0,0,0,-9
292,Z-21686-1_Z-21686-1,Z-21686-1_Z-21686-1,0,0,2,1
293,Z-21746-1_Z-21746-1,Z-21746-1_Z-21746-1,0,0,2,1


In [95]:
Tacera_ccp = Tacera.loc[:,['sample_id','\\RAMAP\\RAMAP\\Clinical Data\\.ACPA/RF Eligibility\\ACPA\\ACPA positive\\']]
Tacera_ccp

,sample_id,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\
0,D-21688-1,Yes
1,D-21679-1,Yes
2,D-21683-1,Yes
3,D-21671-1,Yes
4,D-21673-1,Yes
...,...,...
313,D-22977-1,NaN
314,D-22994-1,NaN
315,D-22998-1,NaN
316,D-22970-1,NaN


In [119]:
Tacera_fam['id'] = Tacera_fam[0].str.split("_",expand=True)[0]
Tacera_fam_ccp = pd.merge(Tacera_fam,Tacera_ccp,how='left',left_on='id',right_on='sample_id')
Tacera_fam_ccp

,0,1,2,3,4,5,id,sample_id,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\
0,D-12945-1_D-12945-1,D-12945-1_D-12945-1,0,0,0,-9,D-12945-1,D-12945-1,NaN
1,D-21657-1_D-21657-1,D-21657-1_D-21657-1,0,0,1,1,D-21657-1,D-21657-1,No
2,D-21658-1_D-21658-1,D-21658-1_D-21658-1,0,0,2,2,D-21658-1,D-21658-1,Yes
3,D-21659-1_D-21659-1,D-21659-1_D-21659-1,0,0,2,2,D-21659-1,D-21659-1,Yes
4,D-21660-1_D-21660-1,D-21660-1_D-21660-1,0,0,2,1,D-21660-1,D-21660-1,Yes
...,...,...,...,...,...,...,...,...,...
290,D-23461-1_D-23461-1,D-23461-1_D-23461-1,0,0,0,-9,D-23461-1,NaN,NaN
291,D-33052-1_D-33052-1,D-33052-1_D-33052-1,0,0,0,-9,D-33052-1,D-33052-1,NaN
292,Z-21686-1_Z-21686-1,Z-21686-1_Z-21686-1,0,0,2,1,Z-21686-1,Z-21686-1,Yes
293,Z-21746-1_Z-21746-1,Z-21746-1_Z-21746-1,0,0,2,1,Z-21746-1,Z-21746-1,Yes


In [120]:
Tacera_fam_ccp['\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\\'] = Tacera_fam_ccp['\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\\'].fillna(-9)
Tacera_fam_ccp['ccp'] = Tacera_fam_ccp['\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\\'].map({'No':1,'Yes':2,-9:-9})
Tacera_fam_ccp

,0,1,2,3,4,5,id,sample_id,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\,ccp
0,D-12945-1_D-12945-1,D-12945-1_D-12945-1,0,0,0,-9,D-12945-1,D-12945-1,-9,-9
1,D-21657-1_D-21657-1,D-21657-1_D-21657-1,0,0,1,1,D-21657-1,D-21657-1,No,1
2,D-21658-1_D-21658-1,D-21658-1_D-21658-1,0,0,2,2,D-21658-1,D-21658-1,Yes,2
3,D-21659-1_D-21659-1,D-21659-1_D-21659-1,0,0,2,2,D-21659-1,D-21659-1,Yes,2
4,D-21660-1_D-21660-1,D-21660-1_D-21660-1,0,0,2,1,D-21660-1,D-21660-1,Yes,2
...,...,...,...,...,...,...,...,...,...,...
290,D-23461-1_D-23461-1,D-23461-1_D-23461-1,0,0,0,-9,D-23461-1,NaN,-9,-9
291,D-33052-1_D-33052-1,D-33052-1_D-33052-1,0,0,0,-9,D-33052-1,D-33052-1,-9,-9
292,Z-21686-1_Z-21686-1,Z-21686-1_Z-21686-1,0,0,2,1,Z-21686-1,Z-21686-1,Yes,2
293,Z-21746-1_Z-21746-1,Z-21746-1_Z-21746-1,0,0,2,1,Z-21746-1,Z-21746-1,Yes,2


In [121]:
Tacera_fam_ccp.loc[:,[0,1,2,3,4,'ccp']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/CCP_assoc/Tacera_ccp.fam",sep='\t',index=False,header=False)

### EAC

In [104]:
EAC_medical = openfile("/exports/reum/knevel_lab/EAC_PatDat/MedicatieVoorschrift_EAC.csv",sep='|')
EAC_medical

/tmp/ipykernel_1809188/631536205.py:1: DtypeWarning: Columns (3,10,11,26) have mixed types.Specify dtype option on import or set low_memory=False.
  EAC_medical = openfile("/exports/reum/knevel_lab/EAC_PatDat/MedicatieVoorschrift_EAC.csv",sep='|')


,VoorschriftID,VoorschriftBron,PseudoID,MedicatieProductID,VoorschrijverID,VoorschrijverNaam,Voorschrijftype,VoorschriftDatumTijd,SchemaOmschrijving,VoorschriftMedicatieStatusCode,VoorschriftRedenVanStoppen,VoorschriftRedenVanOnderbreken,VoorschriftGroepscode,SchemaStartdag,SchemaStartDatumtijd,SchemaEindDatumtijd,SchemaGebruiksduur_Waarde,Gebruiksduur_Eenheid,SchemaFrequentie_Waarde,frequentie_Eenheid,SchemaInterval_Waarde,interval_Eenheid,SchemaZonodig,IndicatieDuurGeneesmiddelCode,ZiekenhuisCode,IndicatieDuurGeneesmiddelOmschrijving,toedieningsHoeveelheid,toedieningsHoeveelheidEenheid,MedicatieKNMPNummer,MedicatieATCCode,MedicatieHPKCode,MedicatiePRKCode,MedicatieGPKCode,MedicatieGPKOmschrijving,MedicatieProductNaam,MedicatieFarmaceutischeVorm,ToedieningsCode,ToedieningsOmschrijving,VoorschriftAfdeling,patient_id
0,13691225,HIX,1237358,0000172181,P32767,D Mohabati,Poliklinisch,2019-04-06 00:00:00,3 x per dag 1 druppel,NaN,NaN,NaN,NaN,1,2019-04-06 00:00:00,None,0.0,None,3.0,D,0.0,None,False,None,None,None,1.0,Druppel,NaN,S01EA03,NaN,43273.0,97101.0,APRACLONIDINE,APRACLONIDINE OOGDRUPPELS 5MG/ML FL 5ML,0,999943,"OOG,LINKS",NaN,5408011
1,13691115,HIX,1314385,0000169477,S01000,Traumatologie,Klinisch,2019-04-06 00:00:00,1 x per dag 10 milligram [adhoc],NaN,NaN,NaN,NaN,1,2019-04-06 08:29:00,2019-04-06 08:29:00,0.0,None,1.0,D,0.0,None,False,None,None,None,10.0,Milligram,NaN,A03FA01,NaN,108.0,71706.0,METOCLOPRAMIDE,METOCLOPRAMIDE INJVLST 5MG/ML (HCL) AMP 2ML,0,5,IV,NaN,5968050
2,13691087,HIX,1314385,0000629699,AB5057,R Blanken,Geïmporteerd / Conversie,2018-11-27 00:00:00,1 x per dag 1 stuk,NaN,NaN,NaN,NaN,1,2018-11-27 00:00:00,2019-03-13 00:00:00,106.0,D,1.0,D,0.0,None,False,None,None,None,1.0,Stuk,16031962.0,H03AA01,636940.0,30171.0,77801.0,LEVOTHYROXINE,"THYRAX DUOTAB TABLET 0,150MG",0,9,ORAAL,NaN,5968050
3,13691086,HIX,1314385,0000595431,AB5057,R Blanken,Geïmporteerd / Conversie,2018-11-27 00:00:00,1 x per dag 1 stuk,NaN,NaN,NaN,NaN,1,2018-11-27 00:00:00,2019-03-13 00:00:00,106.0,D,1.0,D,0.0,None,False,None,None,None,1.0,Stuk,15822761.0,N02BE01,2282321.0,81442.0,125822.0,PARACETAMOL,PARACETAMOL SANDOZ TABLET 1000MG,0,9,ORAAL,NaN,5968050
4,13691085,HIX,1314385,0000557991,AB5057,R Blanken,Geïmporteerd / Conversie,2018-11-27 00:00:00,1 x per dag 1 stuk,NaN,NaN,NaN,NaN,1,2018-11-27 00:00:00,2019-03-13 00:00:00,106.0,D,1.0,D,0.0,None,False,None,None,None,1.0,Stuk,15638723.0,L04AA13,2143046.0,53619.0,109290.0,LEFLUNOMIDE,LEFLUNOMIDE SANDOZ TABLET FILMOMHULD 20MG,0,9,ORAAL,NaN,5968050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73297,75995,HIX,182742,8676,Chipsoft (Conversie),Chipsoft (Conversie),Geïmporteerd / Conversie,2011-03-30 00:00:00,1 x per dag 5 milligram,NaN,NaN,NaN,NaN,1,2011-03-30 00:00:00,None,0.0,None,1.0,D,0.0,None,False,None,None,None,5.0,Milligram,13921339.0,H02AB06,858102.0,30449.0,23809.0,PREDNISOLON,PREDNISOLON TEVA TABLET 5MG,0,9,ORAAL,NaN,74182
73298,75993,HIX,182742,25103,Chipsoft (Conversie),Chipsoft (Conversie),Geïmporteerd / Conversie,2011-03-30 00:00:00,Per week 15 milligram,NaN,NaN,NaN,NaN,1,2011-03-30 00:00:00,None,0.0,None,1.0,D,6.0,D,False,None,None,None,15.0,Milligram,14571013.0,L04AX03,1421778.0,141631.0,16918.0,METHOTREXAAT,"METHOTREXAAT PCH TABLET 2,5MG",0,9,ORAAL,NaN,74182
73299,75991,HIX,182742,8676,Chipsoft (Conversie),Chipsoft (Conversie),Geïmporteerd / Conversie,2011-03-30 00:00:00,1 x per dag 10 milligram,NaN,NaN,NaN,NaN,1,2011-03-30 00:00:00,None,0.0,None,1.0,D,0.0,None,False,None,None,None,10.0,Milligram,13921339.0,H02AB06,858102.0,30449.0,23809.0,PREDNISOLON,PREDNISOLON TEVA TABLET 5MG,0,9,ORAAL,NaN,74182
73300,75989,HIX,182742,81659,Chipsoft (Conversie),Chipsoft (Conversie),Geïmporteerd / Conversie,2011-03-23 00:00:00,1 x per dag 40 milligram,NaN,NaN,NaN,NaN,1,2011-03-23 00:00:00,None,0.0,None,1.0,D,0.0,None,False,None,None,None,40.0,Milligram,15460177.0,A02BC02,2002922.0,4

In [106]:
search(EAC_medical.columns,['ccp','acpa'])

100%|███████████████████████████████████████| 40/40 [00:00<00:00, 313592.82it/s]


[]

### Manchester,RAMS2

In [115]:
Manchester = openfile("/exports/reum/CKe/Newdata/clinical/pheno_192Samples_05.11.2021.csv")
Manchester = Manchester.drop_duplicates('study_id',keep='last')
Manchester

,study_id,fupno,tjc28_value,sjc28_value,crp_value,sex,age_consent,DAS28_baseline,DAS28_6months,antiCCP
1,RAMS01051,6-months,1,0,0.57,female,56,3.697350,1.944626,pos
3,RAMS02026,6-months,3,4,26.50,male,61,4.713943,4.145361,neg
5,RAMS02046,6-months,8,0,1.28,female,44,4.119933,3.218685,neg
7,RAMS02056,6-months,2,0,10.53,female,68,4.394882,2.989357,neg
9,RAMS02078,6-months,2,3,4.74,female,48,4.729321,3.246621,pos
...,...,...,...,...,...,...,...,...,...,...
375,RAMS14016,6-months,1,0,2.67,female,65,3.463230,2.224876,NaN
377,RAMS15003,6-months,0,2,0.80,female,61,5.651514,1.778743,neg
379,RAMS20003,6-months,1,1,4.38,male,69,2.639811,2.656345,pos
381,RAMS20022,6-months,28,2,9.10,female,68,5.599650,5.424985,neg


In [108]:
fam_manchester = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged.fam",sep='\s+',header=None)
fam_manchester

,0,1,2,3,4,5
0,1_RAMS23006,1_RAMS23006,0,0,1,2
1,2_RAMS30004,2_RAMS30004,0,0,2,1
2,7_RAMS20003,7_RAMS20003,0,0,1,1
3,9_RAMS19034,9_RAMS19034,0,0,2,1
4,13_RAMS10012,13_RAMS10012,0,0,2,1
...,...,...,...,...,...,...
183,548_RAMS12114,548_RAMS12114,0,0,2,2
184,553_RAMS07005,553_RAMS07005,0,0,2,2
185,558_RAMS07027,558_RAMS07027,0,0,2,1
186,564_RAMS03012,564_RAMS03012,0,0,2,1


In [109]:
fam_manchester['id'] = fam_manchester[0].str.split("_",expand=True)[1]
fam_manchester

,0,1,2,3,4,5,id
0,1_RAMS23006,1_RAMS23006,0,0,1,2,RAMS23006
1,2_RAMS30004,2_RAMS30004,0,0,2,1,RAMS30004
2,7_RAMS20003,7_RAMS20003,0,0,1,1,RAMS20003
3,9_RAMS19034,9_RAMS19034,0,0,2,1,RAMS19034
4,13_RAMS10012,13_RAMS10012,0,0,2,1,RAMS10012
...,...,...,...,...,...,...,...
183,548_RAMS12114,548_RAMS12114,0,0,2,2,RAMS12114
184,553_RAMS07005,553_RAMS07005,0,0,2,2,RAMS07005
185,558_RAMS07027,558_RAMS07027,0,0,2,1,RAMS07027
186,564_RAMS03012,564_RAMS03012,0,0,2,1,RAMS03012


In [116]:
Manchester_merged = pd.merge(fam_manchester,Manchester,how='left',left_on='id',right_on="study_id")
Manchester_merged

,0,1,2,3,4,5,id,study_id,fupno,tjc28_value,sjc28_value,crp_value,sex,age_consent,DAS28_baseline,DAS28_6months,antiCCP
0,1_RAMS23006,1_RAMS23006,0,0,1,2,RAMS23006,RAMS23006,6-months,1,1,1.38,male,72,5.448171,2.417372,pos
1,2_RAMS30004,2_RAMS30004,0,0,2,1,RAMS30004,RAMS30004,6-months,20,10,1.09,female,74,3.698328,4.831751,NaN
2,7_RAMS20003,7_RAMS20003,0,0,1,1,RAMS20003,RAMS20003,6-months,1,1,4.38,male,69,2.639811,2.656345,pos
3,9_RAMS19034,9_RAMS19034,0,0,2,1,RAMS19034,RAMS19034,6-months,18,2,1.82,female,56,6.063876,4.609592,NaN
4,13_RAMS10012,13_RAMS10012,0,0,2,1,RAMS10012,RAMS10012,6-months,4,4,1.67,female,43,5.700307,3.218903,pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,548_RAMS12114,548_RAMS12114,0,0,2,2,RAMS12114,RAMS12114,6-months,0,1,0.58,female,65,3.798805,1.639140,pos
184,553_RAMS07005,553_RAMS07005,0,0,2,2,RAMS07005,RAMS07005,6-months,0,0,2.50,female,56,3.979376,1.646094,neg
185,558_RAMS07027,558_RAMS07027,0,0,2,1,RAMS07027,RAMS07027,6-months,9,2,30.05,female,30,5.057907,4.586477,neg
186,564_RAMS03012,564_RAMS03012,0,0,2,1,RAMS03012,RAMS03012,6-months,3,8,6.40,female,70,4.100726,3.880685,neg


In [117]:
Manchester_merged['antiCCP'] = Manchester_merged['antiCCP'].fillna(-9)
Manchester_merged['CCP'] = Manchester_merged['antiCCP'].map({'neg':1,'pos':2,-9:-9})

In [118]:
Manchester_merged.loc[:,[0,1,2,3,4,'CCP']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/CCP_assoc/Manchester_ccp.fam",sep='\t',index=False,header=False)